In [1]:
!pip list


Package                 Version
----------------------- -----------
appnope                 0.1.4
asttokens               3.0.0
attrs                   25.3.0
certifi                 2025.7.14
comm                    0.2.3
debugpy                 1.8.17
decorator               5.2.1
executing               2.2.1
h11                     0.16.0
idna                    3.10
ipykernel               7.1.0
ipython                 9.6.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
jupyter_client          8.6.3
jupyter_core            5.9.1
matplotlib-inline       0.2.1
nest-asyncio            1.6.0
outcome                 1.3.0.post0
packaging               25.0
parso                   0.8.5
pexpect                 4.9.0
pip                     25.1.1
platformdirs            4.5.0
prompt_toolkit          3.0.52
psutil                  7.1.2
ptyprocess              0.7.0
pure_eval               0.2.3
Pygments                2.19.2
PySocks                 1.7.1
python-dateutil  